## 請先登入到 DitigalOcean

### https://cloud.digitalocean.com


## 建立 Access Token

點選 API

點選 Generate New Token

![Alt text](https://lh5.googleusercontent.com/YmnhB7UTVa1qDpVsui1IKx3t_dgx4pXoPxb0PW1B94JaOUBlph6BeZ-4xfg_CCQCKebOPh9tjt2cXKnhrK_LgAVyUiDgx9dugyT-7WQz13NyRLpTdxkHgiw3NHe7N30txz4RDVEs)


輸入 Token 名稱

點選 Generate Token

![Alt text](https://lh4.googleusercontent.com/lpN3WJbwY5JNIRyCsPu2PigGpYr0GSpRmaAPlowHA87ShG03Khx482IIv_IABriSBdea8EY_CGMUvSggs0q56YX9HtJM2q_vrbQ18qRNYCQJVOLwERwxQyhSKGanMiKyvJHNNHY4)


接下來就會看到剛剛建立的 token

![Alt text](https://lh4.googleusercontent.com/8LFTtyYojmv_zDW_NVnPCm1soJDo5Z26RyGIp5UfbGiPsNDGDJTP89psxUUClA_CfgOzjGTQgaTykkmPjQzyMaSItfZUs_YeIZfZu69RDW7wF_mEvJBxIGYFTwEnYdqkbi21ftHE)


### Token 只會出現一次 ~~ 不會回來, 記得複製下來 ( 不然就重新建立一個 XD )



#### 以下的指令, 在 docker 裏面使用, 如果是使用 play with docker

查詢 CONTAINERID 指令 docker ps

使用 docker  exec  it  CONTAIENRID  /bin/bash 進入容器內的命令列

## 先來裝 後面需要的套件吧, 請將下面的指令複製貼上到命令列上面（Ctrl + Shift + V）


zypper  -n  install curl wget python-pip jq

## DigitalOcean 需要dopy套件 還有其他的, 一起來安裝

pip  install   dopy

升級 pip

pip  install --upgrade  pip

使用 pip 安裝 ansible

pip2  install ansible

確認一下 ansible 的資訊

ansible   --version

## 開始相關測試

export TOKEN=剛剛建立的TOKEN

### 列出 action

curl  -X  GET "https://api.digitalocean.com/v2/actions" -H "Authorization: Bearer $TOKEN"

### 但是我覺得這樣的方式可讀性比較低, 所以我是搭配 jq
curl  -X  GET "https://api.digitalocean.com/v2/actions" -H "Authorization: Bearer $TOKEN"  |  jq

### 列出 Droplets

把剛剛的 actions 換成 droplets 就可以

curl  -X  GET "https://api.digitalocean.com/v2/droplets" -H "Authorization: Bearer $TOKEN"  |  jq


### 建立 Droplet

curl  -X  POST "https://api.digitalocean.com/v2/droplets" -d '{"name":"My-Droplet","region":"nyc3","size":"512mb","image":"ubuntu-14-04-x64"}' -H "Authorization: Bearer $TOKEN"  -H  "Content-Type: application/json" | jq

Droplet API 的參數用法要看這邊

https://developers.digitalocean.com/documentation/v2/#create-a-new-droplet 


## 可以回到 DigitalOcean dashboard 觀察

### 刪除 Droplets

那如果要刪除某一個  Droplet, 就要先去查詢他的 droplet id

可以用剛剛的方式加上 grep 來列

curl  -X  GET "https://api.digitalocean.com/v2/droplets" -H "Authorization: Bearer $TOKEN"  |  grep '"id":'


然後就是 curl 使用 DELETE 加上 droplet 的 id

### 例如某一個 droplet id 是 88444260, 就會像下面這樣


curl  -X  DELETE "https://api.digitalocean.com/v2/droplets/88444260"  -H "Authorization: Bearer $TOKEN"  -H  "Content-Type: application/json" | jq

## 可以回到 DigitalOcean dashboard 觀察

============================================================================ 


## 接下來實驗 ansible playbook with DigitalOcean

抓個 playbook 來看看

wget https://raw.githubusercontent.com/sakanamax/LearnAnsible/master/playbook/digital_ocean_playbook/create_do_vm.yml

wget https://raw.githubusercontent.com/sakanamax/LearnAnsible/master/playbook/digital_ocean_playbook/delete_do_vm.yml

## 測試 1: 建立 單一 Droplet

ansible-playbook   create_do_vm.yml 



In [ ]:
 [WARNING]: Unable to parse /etc/ansible/hosts as an inventory source


 [WARNING]: No inventory was parsed, only implicit localhost is available


 [WARNING]: provided hosts list is empty, only localhost is available. Note that the implicit localhost does not match 'all'


Enter Digital Ocean API Token: 貼上你的token
Enter Droplet's name [test01]: 
Enter region name [sgp1]: 
Enter image name [ubuntu-16-04-x64]: 


PLAY [use when conditional and setup module] ********************



這邊就是把 TOKEN 貼上, 然後輸入 Droplet 名稱, 區域還有 image 名稱就好, 我預設是 2gb

## 測試 2: 刪除 Droplets

ansible-playbook   delete_do_vm.yml 

In [ ]:
[WARNING]: provided hosts list is empty, only localhost is available. Note that the implicit localhost does not match 'all'

Enter Digital Ocean API Token: 貼上你的token
Enter Droplet's name [test01]:

PLAY [use when conditional and setup module] ***********************************************************************************************************************************************

TASK [Gathering Facts] *********************************************************************************************************************************************************************
ok: [localhost]

TASK [delete digital ocean droplet] ********************************************************************************************************************************************************
changed: [localhost]

PLAY RECAP *********************************************************************************************************************************************************************************
localhost                  : ok=2    changed=1    unreachable=0    failed=0

# 最後別忘記到 DigitalOcean 刪掉 token